In [1]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
import os
import shutil
import numpy as np

source = './data/images/images'
dest = './data/test'

dirs = os.listdir(source)

for d in dirs:
    files = os.listdir(source + '/' + d)
    for f in files:
        if(np.random.rand(1) < 0.1):
            os.makedirs(os.path.dirname(dest + '/' + d + '/' + f), exist_ok = True)
            shutil.move(source + '/' + d + '/' + f, dest + '/' + d + '/' + f)

In [2]:
train_generator = ImageDataGenerator(
    rescale=1/255.0,
    zoom_range=0.15,
    rotation_range=15,
    height_shift_range=0.15,
    width_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=True
)
test_generator = ImageDataGenerator(
    rescale=1/255.0
)

training_data = train_generator.flow_from_directory('./data/images/images', class_mode='categorical', batch_size=32, target_size=(227, 227))
test_data = test_generator.flow_from_directory('./data/test', class_mode='categorical', batch_size=32, target_size=(227, 227))

Found 18514 images belonging to 120 classes.
Found 2063 images belonging to 120 classes.


In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Dense, Average

In [6]:
input_shape = Input(shape=(227,227,3))

tower_1 = Conv2D(96, (5, 5))(input_shape)
tower_1 = MaxPooling2D((2,2))(tower_1)
tower_1 = BatchNormalization()(tower_1)

tower_2 = MaxPooling2D((2, 2))(input_shape)
tower_2 = Conv2D(96, (2, 2))(tower_2)
tower_2 = Dropout(0.15)(tower_2)
tower_2 = Conv2D(96, (2, 2))(tower_2)

tower_3 = MaxPooling2D((2, 2))(input_shape)
tower_3 = Conv2D(96, (3, 3))(tower_3)

concat = Average()([tower_1, tower_2, tower_3])

out = MaxPooling2D(3,3)(concat)

tower_1 = Conv2D(64,(4,4), activation='relu')(out)
tower_1 = BatchNormalization()(tower_1)
tower_1 = Conv2D(32,(3,3))(tower_1)
tower_1 = MaxPooling2D(2,2)(tower_1)
tower_1 = Dropout(0.15)(tower_1)
tower_1 = Conv2D(32,(3,3))(tower_1)
tower_1 = MaxPooling2D(2,2)(tower_1)

tower_2 = Conv2D(64,(3,3))(out)
tower_2 = MaxPooling2D(2,2)(tower_2)
tower_2 = Dropout(0.15)(tower_2)
tower_2 = Conv2D(32,(3,3), activation='relu')(tower_2)
tower_2 = MaxPooling2D(2,2)(tower_2)
tower_2 = BatchNormalization()(tower_2)

concat = Average()([tower_1, tower_2])

out = Flatten()(concat)

out = Dense(512, activation='relu')(out)
out = Dropout(0.15)(out)

out = Dense(256, activation='relu')(out)

out = Dense(120, activation='softmax')(out)

model = Model(input_shape, out)

In [7]:
model.compile(optimizer='SGD', metrics=['accuracy'], loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 113, 113, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 223, 223, 96) 7296        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 112, 112, 96) 1248        max_pooling2d_4[0][0]            
____________________________________________________________________________________________

In [ ]:
model.fit_generator(training_data, epochs=500, validation_data = test_data)

Epoch 1/500
579/579 [==============================] - 484s 837ms/step - loss: 4.7895 - accuracy: 0.0145 - val_loss: 4.7377 - val_accuracy: 0.0218
Epoch 2/500
579/579 [==============================] - 497s 859ms/step - loss: 4.6519 - accuracy: 0.0255 - val_loss: 4.6733 - val_accuracy: 0.0257
Epoch 3/500
579/579 [==============================] - 494s 852ms/step - loss: 4.5410 - accuracy: 0.0329 - val_loss: 4.5442 - val_accuracy: 0.0334
Epoch 4/500
579/579 [==============================] - 492s 849ms/step - loss: 4.4725 - accuracy: 0.0379 - val_loss: 4.6442 - val_accuracy: 0.0402
Epoch 5/500
579/579 [==============================] - 490s 846ms/step - loss: 4.4076 - accuracy: 0.0439 - val_loss: 4.0768 - val_accuracy: 0.0402
Epoch 6/500
579/579 [==============================] - 489s 845ms/step - loss: 4.3523 - accuracy: 0.0508 - val_loss: 4.4186 - val_accuracy: 0.0596
Epoch 7/500
579/579 [==============================] - 495s 855ms/step - loss: 4.3001 - accuracy: 0.0546 - val_loss: 4

579/579 [==============================] - 481s 831ms/step - loss: 3.1072 - accuracy: 0.2293 - val_loss: 4.2737 - val_accuracy: 0.1357
Epoch 57/500
579/579 [==============================] - 481s 831ms/step - loss: 3.0892 - accuracy: 0.2365 - val_loss: 3.6963 - val_accuracy: 0.1638
Epoch 58/500
579/579 [==============================] - 485s 838ms/step - loss: 3.0682 - accuracy: 0.2384 - val_loss: 4.3622 - val_accuracy: 0.1401
Epoch 59/500
579/579 [==============================] - 486s 840ms/step - loss: 3.0743 - accuracy: 0.2377 - val_loss: 3.5792 - val_accuracy: 0.0960
Epoch 60/500
579/579 [==============================] - 504s 870ms/step - loss: 3.0462 - accuracy: 0.2402 - val_loss: 3.2067 - val_accuracy: 0.1750
Epoch 61/500
579/579 [==============================] - 485s 838ms/step - loss: 3.0280 - accuracy: 0.2451 - val_loss: 2.9762 - val_accuracy: 0.1459
Epoch 62/500
579/579 [==============================] - 485s 837ms/step - loss: 3.0086 - accuracy: 0.2509 - val_loss: 3.0457 

579/579 [==============================] - 483s 835ms/step - loss: 2.4781 - accuracy: 0.3500 - val_loss: 4.4581 - val_accuracy: 0.2046
Epoch 112/500
579/579 [==============================] - 488s 842ms/step - loss: 2.4506 - accuracy: 0.3554 - val_loss: 4.0158 - val_accuracy: 0.1861
Epoch 113/500
579/579 [==============================] - 503s 868ms/step - loss: 2.4621 - accuracy: 0.3489 - val_loss: 3.4527 - val_accuracy: 0.1774
Epoch 114/500
579/579 [==============================] - 495s 854ms/step - loss: 2.4356 - accuracy: 0.3574 - val_loss: 4.2877 - val_accuracy: 0.2012
Epoch 115/500
579/579 [==============================] - 493s 852ms/step - loss: 2.4215 - accuracy: 0.3605 - val_loss: 2.8610 - val_accuracy: 0.1789
Epoch 116/500
579/579 [==============================] - 494s 853ms/step - loss: 2.4240 - accuracy: 0.3588 - val_loss: 4.0208 - val_accuracy: 0.2123
Epoch 117/500
579/579 [==============================] - 497s 859ms/step - loss: 2.4155 - accuracy: 0.3664 - val_loss: 3

579/579 [==============================] - 513s 886ms/step - loss: 2.0355 - accuracy: 0.4460 - val_loss: 2.9103 - val_accuracy: 0.2254
Epoch 167/500
579/579 [==============================] - 516s 890ms/step - loss: 2.0342 - accuracy: 0.4404 - val_loss: 4.0882 - val_accuracy: 0.2181
Epoch 168/500
579/579 [==============================] - 529s 913ms/step - loss: 2.0168 - accuracy: 0.4495 - val_loss: 4.2517 - val_accuracy: 0.1997
Epoch 169/500
579/579 [==============================] - 507s 876ms/step - loss: 2.0170 - accuracy: 0.4475 - val_loss: 3.6238 - val_accuracy: 0.2269
Epoch 170/500
579/579 [==============================] - 516s 891ms/step - loss: 2.0047 - accuracy: 0.4497 - val_loss: 3.8643 - val_accuracy: 0.2138
Epoch 171/500
579/579 [==============================] - 515s 889ms/step - loss: 2.0070 - accuracy: 0.4512 - val_loss: 6.0600 - val_accuracy: 0.2302
Epoch 172/500
579/579 [==============================] - 516s 891ms/step - loss: 2.0062 - accuracy: 0.4470 - val_loss: 3

579/579 [==============================] - 517s 893ms/step - loss: 1.7376 - accuracy: 0.5130 - val_loss: 3.3839 - val_accuracy: 0.2424
Epoch 222/500
579/579 [==============================] - 515s 890ms/step - loss: 1.7279 - accuracy: 0.5147 - val_loss: 4.4179 - val_accuracy: 0.2031
Epoch 223/500
579/579 [==============================] - 517s 893ms/step - loss: 1.7236 - accuracy: 0.5185 - val_loss: 3.9742 - val_accuracy: 0.2361
Epoch 224/500
579/579 [==============================] - 515s 889ms/step - loss: 1.7226 - accuracy: 0.5216 - val_loss: 5.5827 - val_accuracy: 0.1983
Epoch 225/500
579/579 [==============================] - 516s 891ms/step - loss: 1.6912 - accuracy: 0.5252 - val_loss: 4.9804 - val_accuracy: 0.2070
Epoch 226/500
579/579 [==============================] - 529s 913ms/step - loss: 1.7035 - accuracy: 0.5200 - val_loss: 4.1919 - val_accuracy: 0.2351
Epoch 227/500
579/579 [==============================] - 604s 1s/step - loss: 1.7031 - accuracy: 0.5217 - val_loss: 4.10